# Taller Bases de datos y Pandas: El dúo dinámico, para el análisis eficiente
Recuerda la VPN está activada para conectarse a MySQL

Crear ambiente virtual
! conda create -n pyladiesmty python=3.9

Activar ambiente virtual
! conda activate pyladiesmty

Archivos extraídos de: https://insideairbnb.com/get-the-data/

In [ ]:
! pip install pymysql cryptography pandas dotenv sqlalchemy

In [5]:
from dotenv import dotenv_values
import pymysql
import pandas as pd
from sqlalchemy import create_engine


In [7]:
config = dotenv_values(".env")

DB_HOST = config["DB_HOST"]
DB_PORT = int(config["DB_PORT"])
DB_USER = config["DB_USER"]
DB_PASS = config["DB_PASS"]
DB_NAME = config["DB_NAME"]
DB_CHAR = config["DB_CHAR"]

In [8]:
timeout = 10
connection = pymysql.connect(
  charset=DB_CHAR,
  connect_timeout=timeout,
  cursorclass=pymysql.cursors.DictCursor,
  db=DB_NAME,
  host=DB_HOST,
  password=DB_PASS,
  read_timeout=timeout,
  port=DB_PORT,
  user=DB_USER,
  write_timeout=timeout,
)

connection_str = f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_str)
path = "data_airbnb_cdmx_2024"

In [ ]:
!ls data_airbnb_cdmx_2024/

## Preparar datos para calendar.csv

In [ ]:
! head data_airbnb_cdmx_2024/raw/calendar.csv

In [ ]:
! wc -l data_airbnb_cdmx_2024/raw/calendar.csv

In [ ]:
df_calendar = pd.read_csv(f"{path}/raw/calendar.csv")
df_calendar.head()

In [ ]:
# Quitar $ y comas, y convertir a float
df_calendar["price"] = (
    df_calendar["price"]
    .astype(str)
    .str.replace("[$,]", "", regex=True)
    .replace("nan", None)
    .astype(float)
)
# Quitar la columna "adjusted_price" no es un dato que usaremos
if "adjusted_price" in df_calendar.columns:
    df_calendar = df_calendar.drop(columns=["adjusted_price"])

df_calendar.head()

In [ ]:
df_calendar.dtypes

In [ ]:
# Como "date" es una palabra reservada en MySQL, renombrar para evitar conflictos
df_calendar = df_calendar.rename(columns={"date": "calendar_date"})
# Transformar en un datatype: datetime
df_calendar["calendar_date"] = pd.to_datetime(df_calendar["calendar_date"], format="%Y-%m-%d")
df_calendar.dtypes

In [ ]:
# Si hay valores vacíos ponerle "f" - false
df_calendar["available"] = df_calendar["available"].replace("", "f")
# Convertir a entero (0 o 1) para que MySQL lo interprete como TINYINT
df_calendar["available"] = df_calendar["available"].map({"t": 1, "f": 0})
df_calendar.dtypes

In [ ]:
# Convertir a enteros, rellenando NaN con 0
df_calendar["minimum_nights"] = pd.to_numeric(df_calendar["minimum_nights"], errors="coerce").fillna(0).astype(int)
df_calendar["maximum_nights"] = pd.to_numeric(df_calendar["maximum_nights"], errors="coerce").fillna(0).astype(int)
df_calendar.dtypes

In [ ]:
df_calendar.to_csv(f"{path}/cleaned/calendars.csv", index=False)

In [55]:
! head data_airbnb_cdmx_2024/cleaned/calendars.csv

listing_id,calendar_date,available,price,minimum_nights,maximum_nights
499553,2024-12-28,0,240.0,3,30
499553,2024-12-29,0,240.0,3,30
499553,2024-12-30,0,240.0,3,30
499553,2024-12-31,0,240.0,3,30
499553,2025-01-01,0,240.0,3,30
499553,2025-01-02,0,240.0,3,30
499553,2025-01-03,1,240.0,3,30
499553,2025-01-04,1,240.0,3,30
499553,2025-01-05,1,240.0,3,30


In [ ]:
# Definir el tamaño de cada chunk (por ejemplo, 10.000 filas)
chunksize = 10000

# Iterar sobre el CSV en chunks
csv_path = f"{path}/cleaned/calendars.csv"

In [ ]:
for i, chunk in enumerate(pd.read_csv(csv_path, chunksize=chunksize)):
    # Transformaciones para cada chunk:
    chunk['listing_id'] = chunk['listing_id'].astype(int)
    chunk['calendar_date'] = pd.to_datetime(chunk['calendar_date'], format='%Y-%m-%d')
    
    try:
        # Siempre usamos "append" ya que la tabla ya existe y tiene la estructura deseada
        with engine.begin() as connection:
            chunk.to_sql("calendars", con=connection, if_exists="append", index=False)
        print(f"Chunk {i} insertado correctamente.")
    except Exception as e:
        print(f"Error en el chunk {i}: {e}")

In [ ]:
# Leer datos con Pandas
df_calendar = pd.read_sql("SELECT * FROM calendars LIMIT 10", engine)
print(df_calendar)

## Preparar datos para listings.csv

In [52]:
! head data_airbnb_cdmx_2024/raw/listings.csv

id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
499553,"Deluxe 4-BR penthouse, views, historic center",1297488,Petra,,Cuauhtémoc,19.43621,-99.13874,Entire home/apt,4985,3,26,2024-10-28,0.17,2,117,1,
501037,The best cheap flat in La Condesa!!,1364145,Humberto,,Cuauhtémoc,19.41578,-99.17425,Entire home/apt,1150,5,134,2024-12-03,0.89,2,232,23,
517503,Nice big room near airport,2547499,Irma Adriana,,Venustiano Carranza,19.43784,-99.07903,Private room,550,1,281,2024-12-11,3.43,1,339,52,
527473,"Nuestro bello departamento, un paraíso en la CDMX.",158585,Fernando,,Cuauhtémoc,19.41887,-99.17333,Entire home/apt,3654,1,354,2024-12-02,2.35,1,216,32,
544105,Polanco great location! Priv Room in Shared Apart,2674950,Elizabeth,,Miguel Hidalgo,19.44289207458496,-99.20138549804688,Private room,1007,2,67,2024-11-26,0.7

In [53]:
! wc -l data_airbnb_cdmx_2024/raw/listings.csv

   26413 data_airbnb_cdmx_2024/raw/listings.csv


In [59]:
import numpy as np
df_listings = pd.read_csv(f"{path}/raw/listings.csv")
df_listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,499553,"Deluxe 4-BR penthouse, views, historic center",1297488,Petra,NaN,Cuauhtémoc,19.436210,-99.138740,Entire home/apt,4985.0,3,26,2024-10-28,0.17,2,117,1,NaN
1,501037,The best cheap flat in La Condesa!!,1364145,Humberto,NaN,Cuauhtémoc,19.415780,-99.174250,Entire home/apt,1150.0,5,134,2024-12-03,0.89,2,232,23,NaN
2,517503,Nice big room near airport,2547499,Irma Adriana,NaN,Venustiano Carranza,19.437840,-99.079030,Private room,550.0,1,281,2024-12-11,3.43,1,339,52,NaN
3,527473,"Nuestro bello departamento, un paraíso en la C...",158585,Fernando,NaN,Cuauhtémoc,19.418870,-99.173330,Entire home/apt,3654.0,1,354,2024-12-02,2.35,1,216,32,NaN
4,544105,Polanco great location! Priv Room in Shared Apart,2674950,Elizabeth,NaN,Miguel Hidalgo,19.442892,-99.201385,Private room,1007.0,2,67,2024-11-26,0.74,1,173,9,NaN


In [60]:
# Preprocesar la columna "price": quitar "$" y comas, convertir a float
if "price" in df_listings.columns:
    df_listings["price"] = df_listings["price"].astype(str).str.replace("[$,]", "", regex=True).replace("", np.nan)
    df_listings["price"] = pd.to_numeric(df_listings["price"], errors="coerce")
df_listings.head()


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,499553,"Deluxe 4-BR penthouse, views, historic center",1297488,Petra,NaN,Cuauhtémoc,19.436210,-99.138740,Entire home/apt,4985.0,3,26,2024-10-28,0.17,2,117,1,NaN
1,501037,The best cheap flat in La Condesa!!,1364145,Humberto,NaN,Cuauhtémoc,19.415780,-99.174250,Entire home/apt,1150.0,5,134,2024-12-03,0.89,2,232,23,NaN
2,517503,Nice big room near airport,2547499,Irma Adriana,NaN,Venustiano Carranza,19.437840,-99.079030,Private room,550.0,1,281,2024-12-11,3.43,1,339,52,NaN
3,527473,"Nuestro bello departamento, un paraíso en la C...",158585,Fernando,NaN,Cuauhtémoc,19.418870,-99.173330,Entire home/apt,3654.0,1,354,2024-12-02,2.35,1,216,32,NaN
4,544105,Polanco great location! Priv Room in Shared Apart,2674950,Elizabeth,NaN,Miguel Hidalgo,19.442892,-99.201385,Private room,1007.0,2,67,2024-11-26,0.74,1,173,9,NaN


In [62]:
# Convertir columnas de fecha a tipo datetime
date_columns = ["last_scraped", "host_since", "calendar_updated", "first_review", "last_review"]
for col in date_columns:
    if col in df_listings.columns:
        df_listings[col] = pd.to_datetime(df_listings[col], errors="coerce", format="%Y-%m-%d")
df_listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,499553,"Deluxe 4-BR penthouse, views, historic center",1297488,Petra,NaN,Cuauhtémoc,19.436210,-99.138740,Entire home/apt,4985.0,3,26,2024-10-28,0.17,2,117,1,NaN
1,501037,The best cheap flat in La Condesa!!,1364145,Humberto,NaN,Cuauhtémoc,19.415780,-99.174250,Entire home/apt,1150.0,5,134,2024-12-03,0.89,2,232,23,NaN
2,517503,Nice big room near airport,2547499,Irma Adriana,NaN,Venustiano Carranza,19.437840,-99.079030,Private room,550.0,1,281,2024-12-11,3.43,1,339,52,NaN
3,527473,"Nuestro bello departamento, un paraíso en la C...",158585,Fernando,NaN,Cuauhtémoc,19.418870,-99.173330,Entire home/apt,3654.0,1,354,2024-12-02,2.35,1,216,32,NaN
4,544105,Polanco great location! Priv Room in Shared Apart,2674950,Elizabeth,NaN,Miguel Hidalgo,19.442892,-99.201385,Private room,1007.0,2,67,2024-11-26,0.74,1,173,9,NaN


In [63]:
# Mapear columnas booleanas: por ejemplo, "host_is_superhost" e "instant_bookable"
bool_columns = ["host_is_superhost", "instant_bookable", "has_availability"]
for col in bool_columns:
    if col in df_listings.columns:
        df_listings[col] = df_listings[col].map({"t": 1, "f": 0}).fillna(0).astype(int)
df_listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,499553,"Deluxe 4-BR penthouse, views, historic center",1297488,Petra,NaN,Cuauhtémoc,19.436210,-99.138740,Entire home/apt,4985.0,3,26,2024-10-28,0.17,2,117,1,NaN
1,501037,The best cheap flat in La Condesa!!,1364145,Humberto,NaN,Cuauhtémoc,19.415780,-99.174250,Entire home/apt,1150.0,5,134,2024-12-03,0.89,2,232,23,NaN
2,517503,Nice big room near airport,2547499,Irma Adriana,NaN,Venustiano Carranza,19.437840,-99.079030,Private room,550.0,1,281,2024-12-11,3.43,1,339,52,NaN
3,527473,"Nuestro bello departamento, un paraíso en la C...",158585,Fernando,NaN,Cuauhtémoc,19.418870,-99.173330,Entire home/apt,3654.0,1,354,2024-12-02,2.35,1,216,32,NaN
4,544105,Polanco great location! Priv Room in Shared Apart,2674950,Elizabeth,NaN,Miguel Hidalgo,19.442892,-99.201385,Private room,1007.0,2,67,2024-11-26,0.74,1,173,9,NaN


In [64]:
# Convertir algunas columnas numericas a enteros
int_columns = ["id", "host_id", "host_listings_count", "host_total_listings_count",
               "minimum_nights", "maximum_nights", "minimum_minimum_nights", 
               "maximum_minimum_nights", "minimum_maximum_nights", "maximum_maximum_nights"]
for col in int_columns:
    if col in df_listings.columns:
        df_listings[col] = pd.to_numeric(df_listings[col], errors="coerce", downcast="integer")
df_listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,499553,"Deluxe 4-BR penthouse, views, historic center",1297488,Petra,NaN,Cuauhtémoc,19.436210,-99.138740,Entire home/apt,4985.0,3,26,2024-10-28,0.17,2,117,1,NaN
1,501037,The best cheap flat in La Condesa!!,1364145,Humberto,NaN,Cuauhtémoc,19.415780,-99.174250,Entire home/apt,1150.0,5,134,2024-12-03,0.89,2,232,23,NaN
2,517503,Nice big room near airport,2547499,Irma Adriana,NaN,Venustiano Carranza,19.437840,-99.079030,Private room,550.0,1,281,2024-12-11,3.43,1,339,52,NaN
3,527473,"Nuestro bello departamento, un paraíso en la C...",158585,Fernando,NaN,Cuauhtémoc,19.418870,-99.173330,Entire home/apt,3654.0,1,354,2024-12-02,2.35,1,216,32,NaN
4,544105,Polanco great location! Priv Room in Shared Apart,2674950,Elizabeth,NaN,Miguel Hidalgo,19.442892,-99.201385,Private room,1007.0,2,67,2024-11-26,0.74,1,173,9,NaN


In [45]:
# Guardar el DataFrame transformado en un nuevo archivo CSV
df_listings.to_csv(f"{path}/cleaned/listings.csv", index=False)

In [65]:
! head data_airbnb_cdmx_2024/cleaned/listings.csv

id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
499553,"Deluxe 4-BR penthouse, views, historic center",1297488,Petra,,Cuauhtémoc,19.43621,-99.13874,Entire home/apt,4985.0,3,26,2024-10-28,0.17,2,117,1,
501037,The best cheap flat in La Condesa!!,1364145,Humberto,,Cuauhtémoc,19.41578,-99.17425,Entire home/apt,1150.0,5,134,2024-12-03,0.89,2,232,23,
517503,Nice big room near airport,2547499,Irma Adriana,,Venustiano Carranza,19.43784,-99.07903,Private room,550.0,1,281,2024-12-11,3.43,1,339,52,
527473,"Nuestro bello departamento, un paraíso en la CDMX.",158585,Fernando,,Cuauhtémoc,19.41887,-99.17333,Entire home/apt,3654.0,1,354,2024-12-02,2.35,1,216,32,
544105,Polanco great location! Priv Room in Shared Apart,2674950,Elizabeth,,Miguel Hidalgo,19.44289207458496,-99.20138549804688,Private room,1007.0,2,67,2024

In [66]:
# Definir el tamaño de cada chunk (por ejemplo, 10.000 filas)
chunksize = 10000

# Iterar sobre el CSV en chunks
csv_path = f"{path}/cleaned/listings.csv"

In [67]:
def convert_bool(x):
    if isinstance(x, str):
        return 1 if x.lower() == 't' else 0
    elif pd.isna(x):
        return 0
    else:
        return int(x)

# Columnas booleanas, de fecha y numericas que se van procesar
bool_columns = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable']
date_columns = ['last_scraped', 'host_since', 'calendar_updated', 'first_review', 'last_review']
numeric_columns = ['scrape_id', 'host_id', 'host_listings_count', 'host_total_listings_count',
                   'accommodates', 'bathrooms', 'bedrooms', 'beds', 'price', 
                   'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 
                   'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights',
                   'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'reviews_per_month']

In [ ]:
for i, chunk in enumerate(pd.read_csv(csv_path, chunksize=chunksize, low_memory=False)):
    print("Chunk", i)
    # Revisar que la columna "id" no tenga valores nulos y convertirla a int64
    if 'id' in chunk.columns:
        chunk = chunk.dropna(subset=['id']) # Elimina filas sin id
        chunk.loc[:, 'id'] = pd.to_numeric(chunk['id'], errors='coerce').astype('int64')
    
    # Procesar columnas booleanas usando .loc
    for col in bool_columns:
        if col in chunk.columns:
            chunk.loc[:, col] = chunk[col].apply(convert_bool)

    for col in date_columns:
        if col in chunk.columns:
            original = chunk[col]
            # Primer intento: conversion sin unidad, usando infer_datetime_format
            conv = pd.to_datetime(original, errors='coerce', infer_datetime_format=True)
            # Identificar valores que NO son nulos originalmente pero se convirtieron en NaT
            invalid = original[original.notna() & conv.isna()]
            if not invalid.empty:
                print(f"Columna '{col}' (sin unidad) tiene {len(invalid)} valores que no se convirtieron: {invalid.unique()}")
            try:
                # Intentamos formatear y reemplazar NaT por None
                conv_str = conv.dt.strftime('%Y-%m-%d')
                conv_str = conv_str.where(pd.notnull(conv_str), None)
                # Si todo sale bien, asignamos la conversión
                chunk.loc[:, col] = conv_str
            except Exception as e:
                print(f"Error formateando la columna '{col}' en el primer intento: {e}")
                # Si falla, se intenta con unit='ns'
                try:
                    conv = pd.to_datetime(original, unit='ns', errors='coerce')
                    invalid = original[original.notna() & conv.isna()]
                    if not invalid.empty:
                        print(f"Columna '{col}' (unit='ns') tiene {len(invalid)} valores que no se convirtieron: {invalid.unique()}")
                    conv_str = conv.dt.strftime('%Y-%m-%d')
                    conv_str = conv_str.where(pd.notnull(conv_str), None)
                    chunk.loc[:, col] = conv_str.astype('string')
                except Exception as e2:
                    print(f"Warning: No se pudo convertir la columna '{col}'. Errores: {e} // {e2}")
                    
    # Convertir otras columnas numericas usando .loc
    for col in numeric_columns:
        if col in chunk.columns:
            chunk.loc[:, col] = pd.to_numeric(chunk[col], errors='coerce')
    
    try:
        # Insertar el chunk en la tabla "listings" usando "append"
        with engine.begin() as connection:
            chunk.to_sql("listings", con=connection, if_exists="append", index=False)
        print(f"Chunk {i} insertado correctamente.")
    except Exception as e:
        print(f"Error en el chunk {i}: {e}")


Chunk 0


/var/folders/3f/2vygd8615pz215mt4l1v091r0000gn/T/ipykernel_64585/1036853346.py:18: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  conv = pd.to_datetime(original, errors='coerce', infer_datetime_format=True)


Chunk 0 insertado correctamente.
Chunk 1


/var/folders/3f/2vygd8615pz215mt4l1v091r0000gn/T/ipykernel_64585/1036853346.py:18: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  conv = pd.to_datetime(original, errors='coerce', infer_datetime_format=True)


Chunk 1 insertado correctamente.
Chunk 2


/var/folders/3f/2vygd8615pz215mt4l1v091r0000gn/T/ipykernel_64585/1036853346.py:18: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  conv = pd.to_datetime(original, errors='coerce', infer_datetime_format=True)


Chunk 2 insertado correctamente.


In [51]:
# Leer datos con Pandas
df_listings = pd.read_sql("SELECT * FROM listings LIMIT 10", engine)
print(df_listings)

       id                                               name  host_id  \
0   35797                                        Villa Dante   153786   
1   44616                                       Condesa Haus   196253   
2   67703                  2 bedroom apt. deco bldg, Condesa   334451   
3   70644   Beautiful light Studio Coyoacan- full equipped !   212109   
4  131610                                     MARIA DEL ALMA   647454   
5  165772      BEST 5 Bedroom HOUSE IN S. Miguel Chapultepec   790208   
6  171109                    Cool room near WTC and Metrobus   816295   
7  180808  Huge Luxurious Suite 70's style, perfectly loc...    36836   
8  187030      Spacious apt with the best rooftop in Condesa   899360   
9  187745           Extraordinarily Spacious Apt. in Condesa   899360   

          host_name neighbourhood_group          neighbourhood  latitude  \
0              Dici                None  Cuajimalpa de Morelos  19.38283   
1          Fernando                None     

## Preparar datos para reviews.csv

In [56]:
! head data_airbnb_cdmx_2024/raw/reviews.csv

listing_id,date
44616,2011-11-09
44616,2012-08-16
44616,2012-12-28
44616,2013-01-04
44616,2013-03-19
44616,2013-03-24
44616,2013-06-15
44616,2013-09-24
44616,2013-11-04


In [57]:
! wc -l data_airbnb_cdmx_2024/raw/reviews.csv

 1278549 data_airbnb_cdmx_2024/raw/reviews.csv


In [77]:
df_reviews = pd.read_csv(f"{path}/raw/reviews.csv")
df_reviews.dtypes

listing_id     int64
date          object
dtype: object

In [78]:
# Convertir columna a datetime y formatear a fecha (YYYY-MM-DD)
df_reviews['review_date'] = pd.to_datetime(df_reviews['date'], errors='coerce').dt.strftime('%Y-%m-%d')
# Reemplazar NaN por None para que se inserten como NULL en la base de datos
df_reviews['review_date'] = df_reviews['review_date'].where(pd.notnull(df_reviews['review_date']), None)

# Eliminar la columna original 'date' si ya no es necesaria
df_reviews = df_reviews.drop(columns=['date'])

df_reviews.dtypes

listing_id      int64
review_date    object
dtype: object

In [81]:
df_reviews.count()

listing_id     1278548
review_date    1278548
dtype: int64

In [83]:
with engine.begin() as connection:
    df_reviews.to_sql("reviews", con=connection, if_exists="append", index=False)

In [84]:
# Leer datos con Pandas
df_calendar = pd.read_sql("SELECT * FROM reviews LIMIT 10", engine)
print(df_calendar)

   id  listing_id review_date
0   1       44616  2011-11-09
1   2       44616  2012-08-16
2   3       44616  2012-12-28
3   4       44616  2013-01-04
4   5       44616  2013-03-19
5   6       44616  2013-03-24
6   7       44616  2013-06-15
7   8       44616  2013-09-24
8   9       44616  2013-11-04
9  10       44616  2013-12-18


In [85]:
# Guardar el DataFrame transformado en un nuevo archivo CSV
df_reviews.to_csv(f"{path}/cleaned/reviews.csv", index=False)